In [1]:
import os
from BiasedErasure.delayed_erasure_decoders.Experimental_Loss_Decoder import *
import time
import numpy as np
from statsmodels.stats.proportion import proportion_confint


distance = 5
output_dir = '/Users/gefenbaranes/Lukin SiV Dropbox/Gefen Baranes/local_Repetitive_QEC/Results/CX_experiment'

decoder_basis = 'XX'
gate_ordering = ['N', 'Z']
num_ancilla_blocks = 4
num_rounds = num_ancilla_blocks + 1
measurement_events = np.load(f'/Users/gefenbaranes/Lukin SiV Dropbox/Gefen Baranes/local_Repetitive_QEC/Oct24/measurement_events_X_NZZrNr_2024_10_06.npy')
num_of_shots = len(measurement_events)

train_measurement_events = measurement_events[::2]
test_measurement_events = measurement_events[1::2]

measurement_events = measurement_events[:100]
# measurement_events = test_measurement_events

print(f"len(measurement_events) = {len(measurement_events)}")

noise_params = {}

Meta_params = {'architecture': 'CBQC', 'code': 'Rotated_Surface', 'logical_basis': decoder_basis,
            'bias_preserving_gates': 'False',
            'noise': 'atom_array', 'is_erasure_biased': 'False', 'LD_freq': '1000', 'LD_method': 'None',
            'SSR': 'True', 'cycles': str(num_rounds - 1),
            'ordering': gate_ordering,
            'decoder': 'MLE',
            'circuit_type': 'logical_CX_2', 'Steane_type': 'None', 'printing': 'True', 'num_logicals': '2',
            'loss_decoder': 'independent',
            'obs_pos': 'd-1', 'n_r': '0'}

print(gate_ordering)

use_loss_decoding = True  # if False: use same DEM every shot, without utilizing SSR.
use_independent_decoder = True  # if False: in every lifecycle, we just apply supercheck at the end. If True: we count the full lifecycle with different potential loss locations and corresponding Clifford propagations.
use_independent_and_first_comb_decoder = False  # This is relevant only if use_independent_decoder=True. If False: use only independent lifecycles. If True: adds a single combination of lifecycles to the decoder.


simulate_data = True
predictions, observable_flips, dems_list = Loss_MLE_Decoder_Experiment(Meta_params, distance, distance, output_dir,
                                                                    measurement_events,
                                                                    None, use_loss_decoding,
                                                                    use_independent_decoder,
                                                                    use_independent_and_first_comb_decoder,
                                                                    simulate_data=simulate_data, logical_gaps=False, 
                                                                    noise_params=noise_params)


logical_probability = np.mean(np.logical_xor(observable_flips, predictions))

# logical_probability = np.mean(np.logical_xor(predictions[:, 0], observable_flips.flatten())).squeeze()
print(r'Probability of logical +1: %.6f' % (1 - logical_probability))

len(measurement_events) = 100
['N', 'Z']
num_CX_per_layer = 1
Using orderings: ['N', 'Z']
generating the Logical circuit took: 1.079396s
Logical circuit that will be used: 
R 2 3 4 5 6 12 13 14 15 16 22 23 24 25 26 32 33 34 35 36 42 43 44 45 46
X_ERROR(0.000337379) 2 3 4 5 6 12 13 14 15 16 22 23 24 25 26 32 33 34 35 36 42 43 44 45 46
I 2 3 4 5 6 12 13 14 15 16 22 23 24 25 26 32 33 34 35 36 42 43 44 45 46
SQRT_Y_DAG 2 4 6 13 15 22 24 26 33 35 42 44 46
PAULI_CHANNEL_1(4.44575e-05, 0.000120258, 7.48173e-05) 2 4 6 13 15 22 24 26 33 35 42 44 46
R 51 52 53 54 55 61 62 63 64 65 71 72 73 74 75 81 82 83 84 85 91 92 93 94 95
X_ERROR(0.000337379) 51 52 53 54 55 61 62 63 64 65 71 72 73 74 75 81 82 83 84 85 91 92 93 94 95
I 51 52 53 54 55 61 62 63 64 65 71 72 73 74 75 81 82 83 84 85 91 92 93 94 95
SQRT_Y_DAG 52 54 61 63 65 72 74 81 83 85 92 94
PAULI_CHANNEL_1(4.44575e-05, 0.000120258, 7.48173e-05) 52 54 61 63 65 72 74 81 83 85 92 94
SQRT_Y 2 3 4 5 6 12 13 14 15 16 22 23 24 25 26 32 33 34 35 36 42 4

ValueError: The circuit contains non-deterministic detectors.
(To allow non-deterministic detectors, use the `allow_gauge_detectors` option.)

This was discovered while analyzing a Z-basis reset (R) on:
    qubit 97

The collapse anti-commuted with these detectors/observables:
    D83

The backward-propagating error sensitivity for D83 was:
    X35
    Z36
    Z45
    X46
    X79
    Z85
    X89
    Z94
    X97

Circuit stack trace:
    during TICK layer #3 of 3
    at instruction #224 [which is R 56 58 60 66 68 70 76 78 80 86 88 90 49 50 57 59 67 69 77 79 87 89 96 97]

In [8]:
a = 'logical_CX'
a[:10]
a[11:]

''

In [ ]:
num_errors = np.mean(np.logical_xor(observable_flips, predictions))
num_errors

0.005